## Model 2


Stochastic optimization model used to solve the two-stage problem

In [299]:
using JuMP,HiGHS,DataFrames
using CSV
using DataFrames

In [300]:
# Import data about buses
df_buses = CSV.read("data/24_buses.csv", DataFrame,types=Dict("I" => String, "C_cur" => String, "Lambda" => String))
#Import demand data
df_demands = CSV.read("data/demand.csv", DataFrame)
# Import the rest of the data
df_other = CSV.read("data/other_data.csv", DataFrame)
# Import the realized wind energy
df_realizedEnergy = CSV.read("data/realizedEnergy.csv", DataFrame)

Row,Node,Q,Omega,energy
,Int64,Int64,Int64,String7
1,1,1,1,50;50
2,1,1,2,10;10


In [301]:
"""
Transforms indices into range for lists of objects (such as generation units)
Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)
Input of missing cells is translated to no values (i.e. empty array)
Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0" return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end


string_to_array

In [302]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for singleNode in eachrow(df_buses)
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => string_to_range(string.(singleNode.I)),
        :Q => string_to_range(string.(singleNode.Q)),
        :J => string_to_range(string.(singleNode.J)),
        :Λ => string_to_range(string.(singleNode.Lambda)),
        :C => string_to_array(string(singleNode.C)) ,
        :C_RU => string_to_array(string(singleNode.C_RU)),
        :C_RD => string_to_array(string(singleNode.C_RD)),
        :C_U => string_to_array(string(singleNode.C_U)),
        :C_D => string_to_array(string(singleNode.C_D)),
        :V_LOL => string_to_array(string(singleNode.V_LOL)),
        :C_cur => string_to_array(string(singleNode.C_cur)),
        :P_max => string_to_array(string(singleNode.P_max)),
        :b => string_to_array(string(singleNode.b)),
        :LC_Max => string_to_array(string(singleNode.LC_Max)),
        :L => Dict(),
        :Ω => 0,
    )    
    # Assign the dictionary to the current node key
    data[singleNode.Node] = data_node
end

#Create dictionary for alle demands dependent on node and load -> array with different times
knownLoads = []
demands = Dict()
for singleNode in eachrow(df_demands)
    data_node = Dict(
        :L => string_to_array(string(singleNode.Demands)),
    )    
    if  singleNode.Node ∉ knownLoads
        demands = Dict()
    end
    push!(knownLoads, singleNode.Node)

    demands[singleNode.Loads] = data_node
    data[singleNode.Node][:L] = demands
    
    data[singleNode.Node][:T] = 1 : singleNode.Periods #string_to_range("1;" , String(singleNode.Periods))
end

#create dictionary with all scenarios
scenarios = Dict()
for singleNode in eachrow(df_other)
    data_node = Dict(
        :π => singleNode.Pi,
    )
    scenarios[singleNode.Omega] = data_node
end

#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
windenergy = Dict()
data_Omega = Dict()
for singleNode in eachrow(df_realizedEnergy)
    data_node = Dict(
        :W_realized => string_to_array(string(singleNode.energy)),
    )
    if ! haskey(windenergy, singleNode.Q)
        data_Omega = Dict()
    end
    data_Omega[singleNode.Omega] = data_node
    windenergy[singleNode.Q] = data_Omega
    data[singleNode.Node][:Windenergy] = windenergy
end

#Float muss in Integer umgewandelt werden
for value in keys(data)
    data[value][:Λ]  = round.(Int, data[value][:Λ])
end

In [303]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0) #voltage angle
    𝛿_scenario=@variable(model, 𝛿_scenario[ω in keys(scenarios),n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0) #voltage angle in case 𝜔
    PF=@variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow
    PF_scenario=@variable(model, PF_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow in case 𝜔
    
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :𝛿 => 𝛿,
        :𝛿_scenario => 𝛿_scenario,
        :PF => PF,
        :PF_scenario => PF_scenario,
    )    
    return vars
end;

In [304]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
    c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]
    
    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][ω, n, ℓ, t] <= data[n][:LC_Max][ℓ]
        
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * data[n][:b][ℓ] == vars[:PF][n, ℓ, t]
    c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:𝛿_scenario][ω, n, ℓ, t] - vars[:𝛿_scenario][ω, ℓ, n, t]) * data[n][:b][ℓ] == vars[:PF_scenario][ω, n, ℓ, t]
    
    #steht im Lehrbuch, aber macht keinen Sinn -> muss auskommentiert bleiben
    #c8[n in keys(data), t in data[n][:T]], vars[:𝛿][n, data[n][:Λ][1], t] == 0
    #c81[ω in keys(scenarios), n in keys(data), t in data[n][:T]], vars[:𝛿_scenario][ω, n, data[n][:Λ][1], t] == 0

    c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:Windenergy][q][ω][:W_realized][t]
    c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]
    

    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:Windenergy][q][ω][:W_realized][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][ω, n, ℓ, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0
    end)
end;

In [305]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] + data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

In [306]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

Min 35 P[2,1,1] + 10 R_U[2,1,1] + 9 R_D[2,1,1] + 35 P[2,1,2] + 10 R_U[2,1,2] + 9 R_D[2,1,2] + 10 P[1,1,1] + 16 R_U[1,1,1] + 15 R_D[1,1,1] + 10 P[1,1,2] + 16 R_U[1,1,2] + 15 R_D[1,1,2] + 30 P[1,2,1] + 13 R_U[1,2,1] + 12 R_D[1,2,1] + 30 P[1,2,2] + 13 R_U[1,2,2] + 12 R_D[1,2,2] + 14 r_U[2,1,2,1] + 14 r_D[2,1,2,1] + 80 L_Shed[2,1,2,1] + 14 r_U[2,1,2,2] + 14 r_D[2,1,2,2] + 80 L_Shed[2,1,2,2] + 21 r_U[2,1,1,1] + 21 r_D[2,1,1,1] + 120 L_Shed[2,1,1,1] + 21 r_U[2,1,1,2] + 21 r_D[2,1,1,2] + 120 L_Shed[2,1,1,2] + 4 r_U[1,1,2,1] + 4 r_D[1,1,2,1] + 12 r_U[1,2,2,1] + 12 r_D[1,2,2,1] + 80 L_Shed[1,1,2,1] + 4 r_U[1,1,2,2] + 4 r_D[1,1,2,2] + 12 r_U[1,2,2,2] + 12 r_D[1,2,2,2] + 80 L_Shed[1,1,2,2] + 6 r_U[1,1,1,1] + 6 r_D[1,1,1,1] + 18 r_U[1,2,1,1] + 18 r_D[1,2,1,1] + 120 L_Shed[1,1,1,1] + 6 r_U[1,1,1,2] + 6 r_D[1,1,1,2] + 18 r_U[1,2,1,2] + 18 r_D[1,2,1,2] + 120 L_Shed[1,1,1,2]
Subject to
 c6[2,1,1] : 𝛿[2,1,1] - 𝛿[1,2,1] - PF[2,1,1] == 0
 c6[2,1,2] : 𝛿[2,1,2] - 𝛿[1,2,2] - PF[2,1,2] == 0
 c6[1,2,1] : -𝛿[2

𝛿[2,1,1] >= 0
 𝛿[2,1,2] >= 0
 𝛿[1,2,1] >= 0
 𝛿[1,2,2] >= 0
 𝛿_scenario[2,2,1,1] >= 0
 𝛿_scenario[2,2,1,2] >= 0
 𝛿_scenario[2,1,2,1] >= 0
 𝛿_scenario[2,1,2,2] >= 0
 𝛿_scenario[1,2,1,1] >= 0
 𝛿_scenario[1,2,1,2] >= 0
 𝛿_scenario[1,1,2,1] >= 0
 𝛿_scenario[1,1,2,2] >= 0


In [307]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
58 rows, 78 cols, 172 nonzeros
44 rows, 58 cols, 136 nonzeros
44 rows, 58 cols, 136 nonzeros
Presolve : Reductions: rows 44(-40); columns 58(-22); elements 136(-64)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.9999944533e+01 Ph1: 6(6); Du: 2(19.9999) 0s
         20     5.3200000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 20
Objective value     :  5.3200000000e+03
HiGHS run time      :          0.00


In [308]:
# Print the values of the variables
println("Variable values:")
println("P:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("P[$n, $i, $t]: ", JuMP.value.(vars[:P][n, i, t]))
end

println("\nR_U:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_U[$n, $i, $t]: ", value(vars[:R_U][n,i, t]))
end

println("\nR_D:")
for n in keys(data), i in data[n][:I], t in data[n][:T]
    println("R_D[$n, $i, $t]: ", value(vars[:R_D][n,i, t]))end

println("\nL_Shed:")
for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
    println("L_Shed[$n, $j, $ω, $t]: ", value(vars[:L_Shed][n,j, ω, t]))
end

println("\nW_spill:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_spill[$n, $q, $ω, $t]: ", value(vars[:W_spill][n,q, ω, t]))
end

println("\nW_s:")
for n in keys(data), q in data[n][:Q], t in data[n][:T]
    println("W_s[$n, $q, $t]: ", value(vars[:W_s][n,q, t]))
end

println("\nPF[n, ℓ, t]:")
for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF[$n, $ℓ, $t]: ", value(vars[:PF][n, ℓ, t]))
end

println("\nPF_scenario[ω, n, ℓ, t]:")
for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
    println("PF_scenario[$ω, $n, $ℓ, $t]: ", value(vars[:PF_scenario][ω, n, ℓ, t]))
end

println("\nr_U[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_U[$n, $i, $ω, $t]: ", value(vars[:r_U][n,i, ω, t]))
end

println("\nr_D[n,i, ω, t]:")
for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
    println("r_D[$n, $i, $ω, $t]: ", value(vars[:r_D][n,i, ω, t]))
end

println("\nData values:")

println("\nW_realized:")
for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
    println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
end

println("\nLoad:")
for n in keys(data), j in data[n][:J], t in data[n][:T]
    println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
end

Variable values:
P:
P[2, 1, 1]: -0.0
P[2, 1, 2]: -0.0
P[1, 1, 1]: 50.0
P[1, 1, 2]: 50.0
P[1, 2, 1]: 40.0
P[1, 2, 2]: 40.0

R_U:
R_U[2, 1, 1]: 40.0
R_U[2, 1, 2]: 40.0
R_U[1, 1, 1]: 0.0
R_U[1, 1, 2]: 0.0
R_U[1, 2, 1]: 0.0
R_U[1, 2, 2]: 0.0

R_D:


R_D[2, 1, 1]: 0.0
R_D[2, 1, 2]: 0.0
R_D[1, 1, 1]: 0.0
R_D[1, 1, 2]: 0.0
R_D[1, 2, 1]: 0.0
R_D[1, 2, 2]: 0.0

L_Shed:
L_Shed[2, 1, 2, 1]: 0.0
L_Shed[2, 1, 2, 2]: 0.0
L_Shed[2, 1, 1, 1]: 0.0
L_Shed[2, 1, 1, 2]: 0.0
L_Shed[1, 1, 2, 1]: 0.0
L_Shed[1, 1, 2, 2]: 0.0
L_Shed[1, 1, 1, 1]: 0.0
L_Shed[1, 1, 1, 2]: 0.0

W_spill:
W_spill[1, 1, 2, 1]: 0.0
W_spill[1, 1, 2, 2]: 0.0
W_spill[1, 1, 1, 1]: 0.0
W_spill[1, 1, 1, 2]: 0.0

W_s:
W_s[1, 1, 1]: 50.0
W_s[1, 1, 2]: 50.0

PF[n, ℓ, t]:
PF[2, 1, 1]: -100.0
PF[2, 1, 2]: -100.0
PF[1, 2, 1]: 100.0
PF[1, 2, 2]: 100.0

PF_scenario[ω, n, ℓ, t]:


PF_scenario[2, 2, 1, 1]: -60.0
PF_scenario[2, 2, 1, 2]: -60.0
PF_scenario[2, 1, 2, 1]: 60.0
PF_scenario[2, 1, 2, 2]: 60.0
PF_scenario[1, 2, 1, 1]: -100.0
PF_scenario[1, 2, 1, 2]: -100.0
PF_scenario[1, 1, 2, 1]: 100.0
PF_scenario[1, 1, 2, 2]: 100.0

r_U[n,i, ω, t]:
r_U[2, 1, 2, 1]: 40.0
r_U[2, 1, 2, 2]: 40.0
r_U[2, 1, 1, 1]: 0.0
r_U[2, 1, 1, 2]: 0.0
r_U[1, 1, 2, 1]: -0.0
r_U[1, 1, 2, 2]: -0.0
r_U[1, 1, 1, 1]: 0.0
r_U[1, 1, 1, 2]: 0.0
r_U[1, 2, 2, 1]: -0.0
r_U[1, 2, 2, 2]: -0.0
r_U[1, 2, 1, 1]: 0.0
r_U[1, 2, 1, 2]: 0.0

r_D[n,i, ω, t]:
r_D[2, 1, 2, 1]: 0.0
r_D[2, 1, 2, 2]: 0.0
r_D[2, 1, 1, 1]: 0.0
r_D[2, 1, 1, 2]: 0.0
r_D[1, 1, 2, 1]: 0.0
r_D[1, 1, 2, 2]: 0.0
r_D[1, 1, 1, 1]: 0.0
r_D[1, 1, 1, 2]: 0.0
r_D[1, 2, 2, 1]: 0.0
r_D[1, 2, 2, 2]: 0.0
r_D[1, 2, 1, 1]: 0.0
r_D[1, 2, 1, 2]: 0.0

Data values:

W_realized:


W_realized[1, 1, 2, 1]: 10.0
W_realized[1, 1, 2, 2]: 10.0
W_realized[1, 1, 1, 1]: 50.0
W_realized[1, 1, 1, 2]: 50.0

Load:
L[2, 1, 1]:100.0
L[2, 1, 2]:100.0
L[1, 1, 1]:40.0
L[1, 1, 2]:40.0
